In [1]:
import requests
import pandas as pd
import io, s3fs, json
from datetime import datetime as dt
pd.set_option('display.max_columns', None)
print('Program run at', dt.now())

Program run at 2021-05-17 12:55:04.398869


This program accesses Child Opportunity Index (COI) data and American Community Survey (ACS) data from the sets' respective APIs. The program formats the datasets as dataframes and saves them to an AWS S3 bucket as CSV files. This program also accesses descriptive header names for ACS data from the ACS API and saves the descriptive headers to the S3 bucket.

# COI  
Access COI raw indicator values from https://data.diversitydatakids.org/dataset/coi20-child-opportunity-index-2-0-database/resource/f16fff12-b1e5-4f60-85d3-3a0ededa30a0.

In [12]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/raw/raw.csv')
coi_rawindicator=pd.read_csv(io.StringIO(r.text), sep=',')
coi_rawindicator.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,ED_APENR,...,HE_SUPRFND,HE_RSEI,SE_POVRATE,SE_PUBLIC,SE_HOME,SE_OCC,SE_MHE,SE_EMPRAT,SE_JOBPROX,SE_SINGLE
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.349463,...,-13.815511,5.933720,9.240363,14.398734,84.335442,27.972027,60321.0,68.187744,5.224787,17.256638
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0.252991,...,-13.815511,7.015026,10.677506,12.068965,75.198936,38.479000,67826.0,82.987556,1.052632,26.556019
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.350239,...,-13.815511,6.083863,10.511090,13.028169,58.568073,40.000000,44039.0,59.521217,1.727862,48.818897
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,0.253207,...,-13.815511,7.306938,22.413794,24.137930,61.941254,30.516430,41287.0,60.339661,0.931315,47.470818
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,0.349768,...,-13.815511,6.162114,11.823454,9.035088,76.315788,31.036932,46002.0,82.875099,1.788375,31.969309


Save raw data to S3.

In [2]:
with open('/Volumes/Key/aws_keys.json') as f:
  creds = json.load(f)

In [3]:
def s3upload(name,df):
    x = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem(key=creds['AWSAccessKeyId'], secret=creds['AWSSecretKey'])
    with fs.open('s3://bleeding-hearts/rawdata/' + name + '.csv', 'wb') as f:
        f.write(x)
    print('saved')

In [15]:
s3upload('coi_rawindicator',coi_rawindicator)

saved


Test opening data from S3 bucket.

In [16]:
test = pd.read_csv('s3://bleeding-hearts/rawdata/coi.csv')

In [17]:
test.head()

,Unnamed: 0,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,...,HE_SUPRFND,HE_RSEI,SE_POVRATE,SE_PUBLIC,SE_HOME,SE_OCC,SE_MHE,SE_EMPRAT,SE_JOBPROX,SE_SINGLE
0,0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,...,-13.815511,5.933720,9.240363,14.398734,84.335442,27.972027,60321.0,68.187744,5.224787,17.256638
1,1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,...,-13.815511,7.015026,10.677506,12.068965,75.198936,38.479000,67826.0,82.987556,1.052632,26.556019
2,2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,...,-13.815511,6.083863,10.511090,13.028169,58.568073,40.000000,44039.0,59.521217,1.727862,48.818897
3,3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,...,-13.815511,7.306938,22.413794,24.137930,61.941254,30.516430,41287.0,60.339661,0.931315,47.470818
4,4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,...,-13.815511,6.162114,11.823454,9.035088,76.315788,31.036932,46002.0,82.875099,1.788375,31.969309


In [50]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/index/index.csv')
coi_index=pd.read_csv(io.StringIO(r.text), sep=',')
coi_index.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,z_ED_nat,...,r_SE_stt,r_COI_stt,c5_ED_met,c5_HE_met,c5_SE_met,c5_COI_met,r_ED_met,r_HE_met,r_SE_met,r_COI_met
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.001198,...,67,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0.008932,...,83,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.013306,...,31,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,-0.003518,...,26,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,-0.014533,...,77,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
coi_index['geoid'].nunique()

144512

In [19]:
s3upload('coi_index',coi_index)

saved


In [20]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/pop/pop.csv')
coi_pop=pd.read_csv(io.StringIO(r.text), sep=',')
coi_pop.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,aian,api,black,hisp,other2,nonwhite,white,total
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0,0,56,0,0,56,368,424
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0,14,19,12,20,65,349,414
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0,33,229,0,24,286,211,497
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,0,10,229,0,18,257,251,508
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,0,15,194,26,0,235,643,878


In [21]:
s3upload('coi_pop',coi_pop)

saved


In [22]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/zscores/zscores.csv')
coi_z=pd.read_csv(io.StringIO(r.text), sep=',')
coi_z.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,zED_APENR,...,zHE_RSEI,zSE_POVRATE,zSE_PUBLIC,zSE_HOME,zSE_OCC,zSE_MHE,zSE_EMPRAT,zSE_JOBPROX,zSE_SINGLE,zSE_ECRES
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.058250,...,0.217542,0.515539,-0.112855,0.823233,-0.391742,-0.084708,-0.689880,0.459048,0.936724,0.157800
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,-0.372527,...,-0.164622,0.393041,0.096836,0.398006,0.334608,0.169833,0.829066,1.065432,0.426814,0.321208
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.061718,...,0.164477,0.407226,0.010503,-0.376019,0.439755,-0.636931,-1.579350,0.967294,-0.793917,-0.027652
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,-0.371563,...,-0.267792,-0.607322,-0.989432,-0.219026,-0.215846,-0.730268,-1.495351,1.083064,-0.719998,-0.672206
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,0.059614,...,0.136821,0.295364,0.369901,0.449986,-0.179864,-0.570353,0.817524,0.958499,0.129990,0.074177


In [23]:
s3upload('coi_z',coi_z)

saved


# ACS  
COI data is based on years 2010 and 2015. To draw direct comparisons, we loaded the ACS 5-year data for both 2010 and 2015.
Load the ACS 5-year DP03 table for 2018.  
Using Data Profiles from https://api.census.gov/data/2018/acs/acs5/profile/variables.html.  

get=group(DP03) based on https://api.census.gov/data/2018/acs/acs5/profile/variables.html  

state numbers: https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [4]:
#open census keys
with open('/Volumes/Key/census_key.txt') as key:
    api_key=key.read().strip()

In [5]:
def get_acs(year,state):
    url=f'https://api.census.gov/data/{year}/acs/acs5/profile?get=group(DP03)&for=tract:*&in=state:{state}&in=county:*&key={api_key}'
    r = requests.get(url)
    if str(r) == '<Response [200]>':
        df = pd.DataFrame(columns=r.json()[0],data=r.json()[1:])
        df['year'] = year
        return df
    else:
        pass

In [6]:
acs = get_acs('2010','01')

In [7]:
acs = acs.append(get_acs('2015','01'))

In [9]:
for i in range(2,56):
    if i <10:
        acs = acs.append(get_acs('2010','0'+str(i)))
        acs = acs.append(get_acs('2015','0'+str(i)))
    else:
        acs = acs.append(get_acs('2010',str(i)))
        acs = acs.append(get_acs('2015',str(i)))
acs

GEO_ID DP03_0001E DP03_0001M DP03_0001PE DP03_0001PM  \
0     1400000US01035960200       1425        266        1425  -888888888   
1     1400000US01035960300       1611        335        1611  -888888888   
2     1400000US01035960400       2851        404        2851  -888888888   
3     1400000US01035960500       1006        269        1006  -888888888   
4     1400000US01035960600       3413        446        3413  -888888888   
...                    ...        ...        ...         ...         ...   
1404  1400000US55079013500       1496        254        1496  -888888888   
1405  1400000US55079016100       2500        204        2500  -888888888   
1406  1400000US55079101100       1570        117        1570  -888888888   
1407  1400000US55079170700       2262        209        2262  -888888888   
1408  1400000US55079005300       1482        108        1482  -888888888   

     DP03_0002E DP03_0002M DP03_0002PE DP03_0002PM DP03_0003E DP03_0003M  \
0           510        170        35.8         9.3        510        170   
1           866        286        53.8         9.3        866        286   
2          1453        350        51.0         8.4       1453        350   
3           545        170        54.2        11.2        545        170   
4          1717        397        50.3         7.4       1717        397   
...         ...        ...         ...         ...        ...        ...   
1404        934        231        62.4         8.7        934        231   
1405       1643        210        65.7         5.6       1643        210   
1406       1138        113        72.5         4.3       1138        113   
1407       1450        180        64.1         5.2       1437        179   
1408       1152        101        77.7         4.0       1152        101   

     DP03_0003PE DP03_0003PM DP03_0004E DP03_0004M DP03_0004PE DP03_0004PM  \
0           35.8         9.3        443        162        31.1         8.8   
1           53.8         9.3        816        286        50.7         9.9   
2           51.0         8.4       1168        299        41.0         8.7   
3           54.2        11.2        488        156        48.5        11.5   
4           50.3         7.4       1534        345        44.9         7.2   
...          ...         ...        ...        ...         ...         ...   
1404        62.4         8.7        758        192        50.7         8.1   
1405        65.7         5.6       1448        197        57.9         5.9   
1406        72.5         4.3       1048        107        66.8         4.4   
1407        63.5         5.1       1346        184        59.5         5.8   
1408        77.7         4.0       1076        113        72.6         4.5   

     DP03_0005E DP03_0005M DP03_0005PE DP03_0005PM DP03_0006E DP03_0006M  \
0            67         61         4.7         4.3          0        119   
1            50         42         3.1         2.7          0        119   
2           285        158        10.0         4.9          0        119   
3            57         72         5.7         6.7          0        119   
4           183        153         5.4         4.2          0        119   
...         ...        ...         ...         ...        ...        ...   
1404        176         91        11.8         5.5          0          9   
1405        195        100         7.8         3.9          0          9   
1406         90         44         5.7         2.8          0          9   
1407         91         54         4.0         2.4         13         19   
1408         76         39         5.1         2.8          0          9   

     DP03_0006PE DP03_0006PM DP03_0007E DP03_0007M DP03_0007PE DP03_0007PM  \
0            0.0         2.3        915        208        64.2         9.3   
1            0.0         2.0        745        148        46.2         9.3   
2            0.0         1.1       1398        274        49.0         8.4   
3            0.0         3.2    

In [10]:
s3upload('acs5',acs)

saved


Define a function to get the headers from the Census website and convert to a dataframe. Run the function for both 2015 and 2010.

In [20]:
def get_headers(year):
    df=requests.get(f'https://api.census.gov/data/{year}/acs/acs5/profile/variables.json')
    d=df.json()
    df2=pd.DataFrame.from_dict(d['variables'])
    return df2

In [21]:
headers_2015 = get_headers('2015')
headers_2015

for  \
label                Census API FIPS 'for' clause   
concept        Census API Geography Specification   
predicateType                            fips-for   
group                                         N/A   
limit                                           0   
predicateOnly                                True   
attributes                                    NaN   
required                                      NaN   
values                                        NaN   

                                               in  \
label                 Census API FIPS 'in' clause   
concept        Census API Geography Specification   
predicateType                             fips-in   
group                                         N/A   
limit                                           0   
predicateOnly                                True   
attributes                                    NaN   
required                                      NaN   
values                                        NaN   

                                                    ucgid  \
label          Uniform Census Geography Identifier clause   
concept                Census API Geography Specification   
predicateType                                       ucgid   
group                                                 N/A   
limit                                                   0   
predicateOnly                                        True   
attributes                                            NaN   
required                                              NaN   
values                                                NaN   

                                                      DP02_0126E  \
label               Estimate!!ANCESTRY!!Total population!!Danish   
concept        SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...   
predicateType                                                int   
group                                                       DP02   
limit                                                          0   
predicateOnly                                                NaN   
attributes                    DP02_0126EA,DP02_0126M,DP02_0126MA   
required                                                     NaN   
values                                                       NaN   

                                                     DP05_0050PE  \
label          Percent!!RACE!!One race!!Native Hawaiian and O...   
concept                    ACS DEMOGRAPHIC AND HOUSING ESTIMATES   
predicateType                                              float   
group                                                       DP05   
limit                                                          0   
predicateOnly                                                NaN   
attributes                 DP05_0050PEA,DP05_0050PM,DP05_0050PMA   
required                                                     NaN   
values                                                       NaN   

                                                      DP04_0047E  \
label          Estimate!!HOUSING TENURE!!Occupied housing uni...   
concept                         SELECTED HOUSING CHARACTERISTICS   
predicateType                                                int   
group                                                       DP04   
limit                                                          0   
predicateOnly                                                NaN   
attributes                    DP04_0047EA,DP04_0047M,DP04_0047MA   
required                                                     NaN   
values                                                       NaN   

                                                   DP02PR_0083PE  \
label          Percent!!RESIDENCE 1 YEAR AGO!!Population 1 ye...   
concept           SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO   
predicateType                                              float   
group                                                     

In [22]:
headers_2010 = get_headers('2010')
headers_2010

for  \
label                Census API FIPS 'for' clause   
concept        Census API Geography Specification   
predicateType                            fips-for   
group                                         N/A   
limit                                           0   
predicateOnly                                True   
attributes                                    NaN   
required                                      NaN   
values                                        NaN   

                                               in  \
label                 Census API FIPS 'in' clause   
concept        Census API Geography Specification   
predicateType                             fips-in   
group                                         N/A   
limit                                           0   
predicateOnly                                True   
attributes                                    NaN   
required                                      NaN   
values                                        NaN   

                                                    ucgid  \
label          Uniform Census Geography Identifier clause   
concept                Census API Geography Specification   
predicateType                                       ucgid   
group                                                 N/A   
limit                                                   0   
predicateOnly                                        True   
attributes                                            NaN   
required                                              NaN   
values                                                NaN   

                                       DP02_0126E  \
label                  Estimate!!ANCESTRY!!Danish   
concept                                       NaN   
predicateType                                 int   
group                                        DP02   
limit                                           0   
predicateOnly                                 NaN   
attributes     DP02_0126M,DP02_0126MA,DP02_0126EA   
required                                      NaN   
values                                        NaN   

                                                     DP05_0050PE  \
label          Percent!!RACE!!One race!!Native Hawaiian and O...   
concept                                                      NaN   
predicateType                                              float   
group                                                       DP05   
limit                                                          0   
predicateOnly                                                NaN   
attributes                 DP05_0050PEA,DP05_0050PM,DP05_0050PMA   
required                                                     NaN   
values                                                       NaN   

                                                      DP04_0047E  \
label          Estimate!!HOUSING TENURE!!Average household si...   
concept                                                      NaN   
predicateType                                              float   
group                                                       DP04   
limit                                                          0   
predicateOnly                                                NaN   
attributes                    DP04_0047EA,DP04_0047M,DP04_0047MA   
required                                                     NaN   
values                                                       NaN   

                                                   DP02PR_0083PE  \
label          Percent!!RESIDENCE 1 YEAR AGO!!Different house...   
concept                                                      NaN   
predicateType                                              float   
group                                                     DP02PR   
limit                                                          0   
predicateOnly                                                NaN   
attr

Upload header dataframes to S3

In [23]:
s3upload('acs_headers_2015',headers_2015)

saved


In [24]:
s3upload('acs_headers_2010',headers_2010)

saved
